1. Implemente ambos métodos para:
a. Modelo de crecimiento logístico
b. Parámetros: r=0.1, K=1000, x₀=10
c. Horizonte de tiempo: 100 unidades

In [2]:
import numpy as np
import time
import pandas as pd

r = 0.1
K = 1000
x0 = 10
T = 100

def f(x):
    return r * x * (1 - x / K)

def analisis(t):
    return K / (1 + ((K - x0) / x0) * np.exp(-r * t))

def metodoEuler(dt):
    steps = int(T / dt)
    x = x0
    trayectoria = [x]
    for _ in range(steps):
        x = x + dt * f(x)
        trayectoria.append(x)
    return np.array(trayectoria)

def metodoRK4(dt):
    steps = int(T / dt)
    x = x0
    trayectoria = [x]
    for _ in range(steps):
        k1 = f(x)
        k2 = f(x + 0.5 * dt * k1)
        k3 = f(x + 0.5 * dt * k2)
        k4 = f(x + dt * k3)
        x = x + dt * (k1 + 2*k2 + 2*k3 + k4) / 6
        trayectoria.append(x)
    return np.array(trayectoria)

2. Analice y compare
a. Error relativo en t=50 para Δt=1.0
b. Threshold entre tiempo de cómputo y precisión
c. Estabilidad con Δt grandes (>5.0

In [ ]:
dt_list = [0.1, 0.5, 1.0, 2.0, 5.0, 10.0]
results = []

for dt in dt_list:
    idx50 = int(50 / dt)

    start = time.perf_counter()
    traj_e = metodoEuler(dt)
    t_e = time.perf_counter() - start

    start = time.perf_counter()
    traj_r = metodoRK4(dt)
    t_r = time.perf_counter() - start

    x50_true = analisis(50)

    err_e = abs(traj_e[idx50] - x50_true) / x50_true * 100
    err_r = abs(traj_r[idx50] - x50_true) / x50_true * 100

    stable_e = traj_e.min() >= 0 and traj_e.max() <= 1.1 * K
    stable_r = traj_r.min() >= 0 and traj_r.max() <= 1.1 * K

    results.append({
        'Δt': dt,
        'Error Euler (%)': err_e,
        'Error RK4 (%)': err_r,
        'Tiempo Euler (s)': t_e,
        'Tiempo RK4 (s)': t_r,
        'Estable Euler': stable_e,
        'Estable RK4': stable_r
    })

df = pd.DataFrame(results)
print(" Resultados Métodos (t=50)")
print(df.to_string(index=False))

row_dt1 = df[df['Δt'] == 1.0].iloc[0]
print(f"Error a t=50 para Δt=1.0 -> Euler: {row_dt1['Error Euler (%)']:.2f}%, RK4: {row_dt1['Error RK4 (%)']:.5f}%")

large_dt = df[df['Δt'] > 5.0]
print("Estabilidad para Δt > 5.0:")
print(large_dt[['Δt', 'Estable Euler', 'Estable RK4']].to_string(index=False))

 Resultados Métodos (t=50)
  Δt  Error Euler (%)  Error RK4 (%)  Tiempo Euler (s)  Tiempo RK4 (s)  Estable Euler  Estable RK4
 0.1         0.639010   8.577765e-09          0.000494        0.001445           True         True
 0.5         3.215440   5.214870e-06          0.000103        0.000314           True         True
 1.0         6.470342   8.061105e-05          0.000044        0.000137           True         True
 2.0        13.017240   1.204259e-03          0.000023        0.000071           True         True
 5.0        31.559292   3.840605e-02          0.000011        0.000029           True         True
10.0        54.152661   4.447222e-01          0.000007        0.000016           True         True
Error a t=50 para Δt=1.0 → Euler: 6.47%, RK4: 0.00008%
Estabilidad para Δt > 5.0:
  Δt  Estable Euler  Estable RK4
10.0           True         True


3. Considere y responda:

a. ¿Qué método mantiene mejor la estabilidad?

Según las estadísticas RK4

b. ¿Cómo escala el error con Δt en cada método?

Euler: O(Δt) y RK4: O(Δt^4) 

c. ¿Cuándo convendría elegir Euler en lugar de RK4?

Cuando se quiere algo más simple, rápido o/con pocos recursos